In [ ]:
import quantopian.pipeline.data.factset.estimates as fe
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import linear_model
from scipy.stats import linregress

from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.pipeline.classifiers.fundamentals import Sector
from quantopian.pipeline.data.factset.estimates import PeriodicConsensus
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.domain import US_EQUITIES
from quantopian.research import run_pipeline
from quantopian.pipeline.filters import QTradableStocksUS, StaticAssets
from quantopian.pipeline.data import Fundamentals

from quantopian.pipeline.factors import SimpleMovingAverage

TICKERS = ['GS']

E6 = 1000000000

mask = QTradableStocksUS() & Sector().notnull()
mask &= (Fundamentals.market_cap.latest > 1000000000)
mask = StaticAssets(symbols(TICKERS))


fq1_eps_cons = PeriodicConsensus.slice('EPS', 'qf', 1)
fq2_eps_cons = PeriodicConsensus.slice('EPS', 'qf', 2)

fq0_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', 0)
fq0_eps_act = fe.Actuals.slice('EPS', 'qf', 0)


fq1_eps_mean = fq1_eps_cons.mean.latest
fq2_eps_mean = fq2_eps_cons.mean.latest

fq0_eps_cons_mean = fq0_eps_cons.mean.latest
fq0_eps_act_value = fq0_eps_act.actual_value.latest


estimated_growth_factor = (fq2_eps_mean - fq1_eps_mean) / fq1_eps_mean
fq0_surprise = (fq0_eps_act_value - fq0_eps_cons_mean) / fq0_eps_cons_mean

close_price = USEquityPricing.close.latest
open_price = USEquityPricing.open.latest
mkt_cap = Fundamentals.market_cap.latest
enterprise_value = Fundamentals.enterprise_value.latest
earnings_growth = Fundamentals.net_income_growth.latest
equity_growth = Fundamentals.equity_per_share_growth.latest
revenue_growth = Fundamentals.revenue_growth.latest
total_debt_equity_ratio = Fundamentals.total_debt_equity_ratio.latest
book_value_yield = Fundamentals.book_value_yield.latest
fcf_yield = Fundamentals.fcf_yield.latest
earning_yield = Fundamentals.earning_yield.latest
forward_earning_yield = Fundamentals.forward_earning_yield.latest
sector = sector = Fundamentals.morningstar_sector_code.latest

ma50 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=50, mask=mask)
ma200 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=200, mask=mask)
ma50_ma200 = ma50 / ma200
price_ma50 = close_price / ma50
price_ma200 = close_price / ma200

pe_ttm = SimpleMovingAverage(inputs=[Fundamentals.pe_ratio], window_length=250, mask=mask)
pe = Fundamentals.pe_ratio.latest
pe_to_pe_ttm = pe / pe_ttm

pipe = Pipeline(
    columns={
        #'fq1_eps_mean': fq1_eps_mean,
        #'fq2_eps_mean': fq2_eps_mean,
        'eps_surprise_factor': fq0_surprise,
        'close': close_price,
        'open': open_price,
        'mkt_cap': mkt_cap / E6,
        'enterprise_value': enterprise_value / E6,
        'earnings_growth': earnings_growth,
        'equity_growth': equity_growth,
        'revenue_growth': revenue_growth,
        'total_debt_equity_ratio': total_debt_equity_ratio,
        'book_value_yield': book_value_yield,
        'fcf_yield': fcf_yield,
        'earning_yield': earning_yield,
        'forward_earning_yield': forward_earning_yield,
        'sector' : sector,
        'ma50_ma200': ma50_ma200,
        'price_ma50': price_ma50,
        'price_ma200': price_ma200,
        'pe_to_pe_ttm'     : pe_to_pe_ttm
        
    },
    domain=US_EQUITIES,
    #screen=estimated_growth_factor.notnull(),
    screen=mask

)

# Run the pipeline over a one year period and print the result.
start_date = '2009-01-01'
end_date = '2019-01-01'
df = run_pipeline(pipe, start_date, end_date)#.head(30)

surprise_diff = df['eps_surprise_factor'].diff()
surprise_diff = surprise_diff.loc[surprise_diff!=0]
#day_before_close = df['close'].shift(1)
day_after_open = df['open'].shift(-1)
day_after_close = df['close'].shift(-1)
week_after_close = df['close'].shift(-6)
#df['day_before_close'] = day_before_close
#df['day_after_open'] = day_after_open
df['day_after_close'] = day_after_close
df['week_after_close'] = week_after_close
# print(df.head())
df['day_after_close_diff'] = (df['day_after_close'] - df['close'])/df['close']
df['week_after_close_diff'] = (df['week_after_close'] - df['close'])/df['close']

df['day_after_impact'] = df['day_after_close_diff'] / df['eps_surprise_factor']
df['week_after_impact'] = df['week_after_close_diff'] / df['eps_surprise_factor']

#sector_df = pd.get_dummies(df['sector'])

#df = pd.concat([df, sector_df])
#print("df")
#print(df.head())

model = linear_model.LinearRegression()

result = df.loc[surprise_diff.index,:].dropna()

# Day after 

features = ['eps_surprise_factor', 
            #'mkt_cap',
            #'book_value_yield',
            #'fcf_yield',
            #'earning_yield',
            #'forward_earning_yield',
            #'revenue_growth',
            #'earnings_growth',
            'ma50_ma200',
            'price_ma50',
            'price_ma200',
            #'total_debt_equity_ratio',
            #'pe_to_pe_ttm'
            #]
           ]# + list(sector_df.columns)

            #'eps_growth', 'equity_growth']
    
def calc_adj_rsquared(model, x, y):
    return 1 - (1-model.score(x, y))*(len(y)-1)/(len(y)-x.shape[1]-1)

x = result[features]

# day after
y = result['day_after_close_diff']
model.fit(x, y)
adj_rsquared = calc_adj_rsquared(model, x, y)
print("day after")
print(list(zip(features, model.coef_)))
print("intercept=",model.intercept_)
print("adj rsquared=", adj_rsquared)

# week after
y = result['week_after_close_diff']
model.fit(x, y)
print("week after")
adj_rsquared = calc_adj_rsquared(model, x, y)
print(list(zip(features, model.coef_)))
print("intercept=",model.intercept_)
print("adj rsquared=", adj_rsquared)
